Bank Account Management System

In [1]:
import pickle  # For saving and loading data
import os  # To check file existence
import random  # For generating account numbers and passwords
import numpy as np  # (Unused, but imported)
import datetime as dt  # For timestamps in transactions

# File names for storing account and transaction data
data_file = 'accounts_data.pkl'
transactions_file = 'transactions_data.pkl'

# Initialize dictionaries to hold account details and transactions
account_details = {}
transactions = {}

# Load data from files if they exist
if os.path.exists(data_file):
    with open(data_file, 'rb') as file:
        account_details = pickle.load(file)

if os.path.exists(transactions_file):
    with open(transactions_file, 'rb') as file:
        transactions = pickle.load(file)

# Function to save account and transaction data to files
def save_data():
    with open(data_file, 'wb') as file:
        pickle.dump(account_details, file)
    with open(transactions_file, 'wb') as file:
        pickle.dump(transactions, file)

# Display welcome message
print("*"*47)
text = "Welcome to WsCube Bank"
padded_text = text.center(47)
print(padded_text)
print("*"*47)

# Main menu loop
while True:
    print("\nMenu Options: ")
    print("-"*13)
    print("""
    Press 1 for Opening an Account
    Press 2 to View Account Details
    Press 3 to Withdraw/Deposit/Transfer
    Press 4 to View Transaction History
    Press 5 to Exit the Program""")
    print("-"*47)
    
    # Get user input for menu choice
    choice = input("Enter your choice between 1-5: ")

    # Validate user input
    if not choice.isdigit() or int(choice) < 1 or int(choice) > 5:
        print("Invalid input. Please enter a number between 1 and 5.")
        continue

    choice = int(choice)
    print("-"*47)

    # Option 1: Open a new account
    if choice == 1:
        pancard = input("Enter Your Pancard Number    : ")
        name = input("Enter your name here         : ")
        account_type = input("Account Type (Savings/Credit): ")
        initial_deposit = float(input("Enter initial deposit        : "))
        account_no = "9876543210" + str(random.randint(1000, 9999))
        password = str(random.randint(100000, 999999))
        
        # Ensure unique account number
        while account_no in [details[3] for details in account_details.values()]:
            account_no = "9876543210" + str(random.randint(1000, 9999))
        
        # Save account details
        details = [name, account_type, initial_deposit, account_no, password]
        account_details[pancard] = details
        
        # Save data to file
        save_data()
        
        # Display account details
        print("-" * 47)
        print(f"Account Created Successfully!")
        print("*" * 32)
        print(f"Account No     : {account_no}")
        print(f"Pin Number     : {password}")
        print(f"Name           : {name}")       
        print(f"Account Type   : {account_type}") 
        print(f"Initial Balance: {initial_deposit}")
        print("*" * 32)

    # Option 2: View account details
    elif choice == 2:  
        ask_account_number = input("Enter Account Number   : ")
        ask_password = input("Enter your 6-Digit Pin : ")
    
        # Check if account exists
        account_found = None
        for pancard, details in account_details.items():
            if details[3] == ask_account_number and details[4] == ask_password:
                account_found = details
                break

        # Display account details if found
        if account_found:  
            print("*" * 47)
            text1 = "Account Details"
            padded_text1 = text1.center(47)
            print(padded_text1)
            print("*" * 47)
            print("Account Holder's Name: ", account_found[0])
            print("Account Type         : ", account_found[1])
            print("Account Number       : ", account_found[3])
            print("Account Balance      : ", account_found[2])
            print("*" * 47)
        else:
            print("Invalid account number or pin number.")

    # Option 3: Withdraw/Deposit/Transfer
    elif choice == 3:
        text2 = "Withdraw - Deposit - Transfer"
        padded_text2 = text2.center(47)
        print(padded_text2)
        print("*"*47)
        print("""
        Press 1 to Withdraw
        Press 2 to Deposit
        Press 3 to Transfer
        """)
        print("-"*47)
        
        n_choice = input("Enter your choice between 1-3: ")
        if not n_choice.isdigit() or int(n_choice) < 1 or int(n_choice) > 3:
            print("Enter a valid option.")
            continue
        n_choice = int(n_choice)
        print("-"*47)

        # Withdraw
        if n_choice == 1:
            ask_account_number = input("Enter Account Number    : ")
            ask_password = input("Enter your 6-Digit Pin  : ")
                                
            # Locate the account
            account_found = None
            for pancard, details in account_details.items():
                if details[3] == ask_account_number and details[4] == ask_password:
                    account_found = pancard
                    break
            
            if account_found: 
                withdraw_amount = float(input("Enter withdrawal amount : "))
                # Check sufficient balance
                if withdraw_amount > account_details[account_found][2]:
                    print("Sorry! Insufficient Balance")
                else:
                    account_details[account_found][2] -= withdraw_amount
                    timestamp = dt.datetime.now()
                    ts = timestamp.strftime("%Y-%m-%d %H:%M:%S")
                    t_data = [withdraw_amount, "Debited", account_details[account_found][2]]

                    # Record the transaction
                    if account_found not in transactions:
                        transactions[account_found] = {}
                    transactions[account_found][ts] = t_data
                    save_data()

                    print("-" * 47)
                    print(f"Withdrawal Successful! Remaining Balance: {account_details[account_found][2]}")
                    print("-" * 47)
                    print("Transaction Details:")
                    print("-" * 33)
                    print(f"Date & Time : {ts}")
                    print(f"Amount      : {withdraw_amount}")
                    print(f"Status      : Debited")
                    print(f"Balance     : {account_details[account_found][2]}")
            else:
                print("Invalid account number or pin.")

        # Deposit
        elif n_choice == 2:
            ask_account_number = input("Enter Account Number   : ")
            ask_password = input("Enter your 6-Digit Pin : ")
                               
            # Locate the account
            account_found = None
            for pancard, details in account_details.items():
                if details[3] == ask_account_number and details[4] == ask_password:
                    account_found = pancard 
                    break

            if account_found: 
                deposit_amount = float(input("Enter deposit amount   : "))
                account_details[account_found][2] += deposit_amount
                timestamp = dt.datetime.now()
                ts = timestamp.strftime("%Y-%m-%d %H:%M:%S")
                t_data = [deposit_amount, "Credited", account_details[account_found][2]]

                # Record the transaction
                if account_found not in transactions:
                    transactions[account_found] = {}
                transactions[account_found][ts] = t_data
                save_data()

                print("-" * 47)
                print(f"Deposit Successful! New Balance: {account_details[account_found][2]}")
                print("-" * 47)
                print("Transaction Details:")
                print("-" * 33)
                print(f"Date & Time : {ts}")
                print(f"Amount      : {deposit_amount}")
                print(f"Status      : Credited")
                print(f"Balance     : {account_details[account_found][2]}")
            else:
                print("Invalid account number or pin.")

        # Transfer
        else:
            ask_account_number = input("Enter Your Account Number         : ")
            ask_password = input("Enter Your 6-Digit Pin            : ")

            # Locate the sender account
            sender_account_found = None
            for pancard, details in account_details.items():
                if details[3] == ask_account_number and details[4] == ask_password:
                    sender_account_found = pancard 
                    break

            if sender_account_found: 
                recipient_account_no = input("Enter Recipient's Account Number  : ")
                transfer_amount = float(input("Enter amount you wish to transfer : "))

                # Locate the recipient account
                recipient_account_found = None
                for pancard, details in account_details.items():
                    if details[3] == recipient_account_no:
                        recipient_account_found = pancard
                        break

                if recipient_account_found:  
                    if transfer_amount > account_details[sender_account_found][2]:
                        print("Sorry! Insufficient Balance")
                    else:
                        account_details[sender_account_found][2] -= transfer_amount
                        account_details[recipient_account_found][2] += transfer_amount
                        
                        timestamp = dt.datetime.now()
                        ts = timestamp.strftime("%Y-%m-%d %H:%M:%S")
                        sender_t_data = [transfer_amount, "Transfer/Debited", account_details[sender_account_found][2]]
                        recipient_t_data = [transfer_amount, "Transfer/Credited", account_details[recipient_account_found][2]]

                        # Record transactions for both accounts
                        if sender_account_found not in transactions:
                            transactions[sender_account_found] = {}
                        transactions[sender_account_found][ts] = sender_t_data

                        if recipient_account_found not in transactions:
                            transactions[recipient_account_found] = {}
                        transactions[recipient_account_found][ts] = recipient_t_data

                        save_data()

                        print("-" * 47)
                        print(f"Transfer Successful! New Balance: {account_details[sender_account_found][2]}")
                        print("-" * 47)
                        print("Transaction Details (Sender):")
                        print("-" * 33)
                        print(f"Date & Time : {ts}")
                        print(f"Transfer To : {recipient_account_no}")
                        print(f"Amount      : {transfer_amount}")
                        print(f"Status      : Transfer/Debited")
                        print(f"Balance     : {account_details[sender_account_found][2]}")
                else:
                    print("Recipient's Account Number not found.")
            else:
                print("Invalid account number or pin.")

    # Option 4: View transaction history
    elif choice == 4:
        ask_account_number = input("Enter Account Number   : ")
        ask_password = input("Enter your 6-Digit Pin : ")

        # Locate the account
        account_found = None
        for pancard, details in account_details.items():
            if details[3] == ask_account_number and details[4] == ask_password:
                account_found = pancard 
                break

        if account_found: 
            print("-" * 64)
            text4 = "Transactions History"
            padded_text4 = text4.center(64)
            print(padded_text4)
            print("-" * 64)
            print("{:<22} {:<12} {:<19} {:<12}".format("Date & Time", "Amount", "Type", "Balance"))
            print("-" * 64)

            # Display transactions if they exist
            if account_found in transactions:
                for timestamp, transaction_details in transactions[account_found].items():
                    print("{:<22} {:<12} {:<19} {:<12}".format(
                        timestamp, transaction_details[0], transaction_details[1], transaction_details[2]
                    ))
            else:
                print("No transactions found for this account.")
        else:
            print("Invalid account number or pin.")
                
    # Option 5: Exit the program
    elif choice == 5:
        print("*"*64)
        text5 = "Thank you for banking with WsCube Bank!"
        padded_text5 = text5.center(64)
        print(padded_text5)
        print("*"*64)
        break


***********************************************
             Welcome to WsCube Bank            
***********************************************

Menu Options: 
-------------

    Press 1 for Opening an Account
    Press 2 to View Account Details
    Press 3 to Withdraw/Deposit/Transfer
    Press 4 to View Transaction History
    Press 5 to Exit the Program
-----------------------------------------------


Enter your choice between 1-5:  6


Invalid input. Please enter a number between 1 and 5.

Menu Options: 
-------------

    Press 1 for Opening an Account
    Press 2 to View Account Details
    Press 3 to Withdraw/Deposit/Transfer
    Press 4 to View Transaction History
    Press 5 to Exit the Program
-----------------------------------------------


Enter your choice between 1-5:  1


-----------------------------------------------


...

-----------------------------------------------


Enter Account Number   :  98765432107877
Enter your 6-Digit Pin :  223633


----------------------------------------------------------------
                      Transactions History                      
----------------------------------------------------------------
Date & Time            Amount       Type                Balance     
----------------------------------------------------------------
2025-01-18 17:50:05    65000.0      Transfer/Debited    35000.0     

Menu Options: 
-------------

    Press 1 for Opening an Account
    Press 2 to View Account Details
    Press 3 to Withdraw/Deposit/Transfer
    Press 4 to View Transaction History
    Press 5 to Exit the Program
-----------------------------------------------


Enter your choice between 1-5:  5


-----------------------------------------------
****************************************************************
            Thank you for banking with WsCube Bank!             
****************************************************************
